# Billboard Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
from fileUtils import getBasename, getDirname, getBaseFilename
from fsUtils import isFile, isDir, moveFile, removeFile
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib

from discogsBase import discogs
from multiArtist import multiArtist
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-15 19:15:07.272261


In [23]:
disc = discogs()
discdata = disc.getArtistVariationNameToIDsData()
discArtists = [x.upper() for x in discdata.keys() if x is not None]
for k in list(discdata.keys()):
    discdata[k.upper()] = discdata[k]
print("Found {0} artist names".format(len(discArtists)))
#discdata = disc.getArtistVariationNameToIDsData()

Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Loading data from /Volumes/Music/Discog/db/VariationNameToIDs.p
  --> This file is 27.5MB.
Loading /Volumes/Music/Discog/db/VariationNameToIDs.p
Found 975256 artist names


# Global Vars

In [6]:
basedir = getcwd()
savedir  = join(basedir, "results")

# Starter Class Section

In [3]:

try:
    filename = glob(join(basedir, "data", "billboard", "starter.html"))[0]
except:
    print("Could not find starter HTML file!")
fdata = getHTML(filename)

In [ ]:
chartData  = {}
dirname = None
baseURL = "https://www.billboard.com"
for iul,ul in enumerate(fdata.findAll("ul")):
    lis = ul.findAll("li", {"class": "chart-group__item"})
    for j,li in enumerate(lis):  
        a = li.find('a')
        if a is not None:
            href = a['href']
            text = a.text
            subdir = getDirname(href)[1:]
            chartData[text] = [baseURL, subdir, getBasename(href)]

# Download Yearly Information

In [ ]:
from time import sleep
years = [str(x) for x in range(1958,2014)]
for year in years:
    url="https://www.billboard.com/archive/charts/{0}".format(year)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    savedir = join(basedir, "data", "billboard", "yearly")
    savename = join(savedir, "{0}.p".format(year))
    if isFile(savename):
        continue
    
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(1)

# Parse Yearly Information

In [ ]:
downloads = {}
baseURL = "https://www.billboard.com"
for ifile in sorted(glob(join(basedir, "data", "billboard", "yearly", "*.p"))):
    year = getBaseFilename(ifile)
    if downloads.get(year) is None:
        downloads[year] = {}
    fdata = getHTML(ifile)
    for iul,ul in enumerate(fdata.findAll("ul")):
        lis = ul.findAll("li", {"class": "chart-group__item"})
        for j,li in enumerate(lis):  
            a = li.find('a')
            if a is not None:
                href = a['href']
                text = a.text
                subdir = getDirname(href)[1:]
                downloads[year][href] = [baseURL, subdir, getBasename(href)]

# Download Category Data

In [ ]:
baseURL = "https://www.billboard.com"

for year, yearData in downloads.items():   
    for href, hrefData in yearData.items():
        url  = "{0}/{1}".format(baseURL, href)
        year = getBasename(hrefData[1])
        category = hrefData[2]

        savedir  = join(basedir, "data", "billboard", "categories")
        savename = join(savedir, "{0}-{1}.p".format(year, category))

        if isFile(savename):
            continue

        print("  Trying to download and save {0}".format(savename))
        try:

            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers={'User-Agent':user_agent,} 

            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need
        except:
            continue

        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(1)        

# Parse Category Data

In [11]:
data = {}

baseURL = "https://www.billboard.com"
names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
categories = set([x[5:] for x in names])

albumCategories  = [x for x in categories if x.endswith("albums")]
songCategories   = [x for x in categories if not x.endswith("albums")]


names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
years = sorted(set([x[:4] for x in names]))

print("There are {0} years".format(len(years)))
print("There are {0} charts".format(len(categories)))
print("There are {0} album charts".format(len(albumCategories)))
print("There are {0} song charts".format(len(songCategories)))

There are 61 years
There are 90 charts
There are 32 album charts
There are 58 song charts


In [10]:
categories

{'adult-contemporary',
 'adult-pop-songs',
 'alternative-albums',
 'alternative-songs',
 'americana-folk-albums',
 'artist-100',
 'billboard-200',
 'bluegrass-albums',
 'blues-albums',
 'canadian-albums',
 'canadian-hot-100',
 'catalog-albums',
 'china-v-chart',
 'christian-airplay',
 'christian-albums',
 'christian-digital-song-sales',
 'christian-songs',
 'christian-streaming-songs',
 'classical-albums',
 'comedy-albums',
 'country-airplay',
 'country-albums',
 'country-digital-song-sales',
 'country-songs',
 'country-streaming-songs',
 'dance-club-play-songs',
 'dance-electronic-albums',
 'dance-electronic-digital-song-sales',
 'dance-electronic-songs',
 'dance-electronic-streaming-songs',
 'digital-albums',
 'digital-song-sales',
 'gospel-airplay',
 'gospel-albums',
 'gospel-digital-song-sales',
 'gospel-songs',
 'gospel-streaming-songs',
 'hard-rock-albums',
 'heatseekers-albums',
 'holiday-albums',
 'holiday-season-digital-song-sales',
 'holiday-songs',
 'holiday-streaming-songs'

['r-b-hip-hop-albums', 'rock-albums', 'latin-pop-albums', 'dance-electronic-albums', 'tastemaker-albums', 'latin-albums', 'world-albums', 'vinyl-albums', 'heatseekers-albums', 'alternative-albums', 'blues-albums', 'catalog-albums', 'gospel-albums', 'christian-albums', 'bluegrass-albums', 'new-age-albums', 'country-albums', 'regional-mexican-albums', 'independent-albums', 'comedy-albums', 'digital-albums', 'canadian-albums', 'rap-albums', 'classical-albums', 'jazz-albums', 'reggae-albums', 'tropical-albums', 'r-and-b-albums', 'holiday-albums', 'hard-rock-albums', 'kids-albums', 'americana-folk-albums']
['adult-pop-songs', 'tropical-songs', 'radio-songs', 'billboard-200', 'latin-pop-songs', 'hot-christian-songs', 'latin-songs', 'latin-streaming-songs', 'digital-song-sales', 'country-streaming-songs', 'pop-songs', 'hot-mainstream-rock-tracks', 'twitter-emerging-artists', 'canadian-hot-100', 'holiday-streaming-songs', 'alternative-songs', 'soundtracks', 'jazz-songs', 'china-v-chart', 'r-an

In [13]:
dict(zip([1,2],[3,4]))

{1: 3, 2: 4}

In [15]:
data = {}

for categoryType,categoryData in dict(zip(["album", "song"], [albumCategories, songCategories])).items():
    for category in sorted(categoryData):
        categoryData = {}
        for year in sorted(years):
            for ifile in sorted(glob(join(basedir, "data", "billboard", "categories", "{0}-{1}.p".format(year, category)))):
                try:
                    fdata = getHTML(ifile)
                except:
                    removeFile(ifile, debug=True)
                    continue
                for i,table in enumerate(fdata.findAll('table')):
                    ths = table.findAll("th")
                    trs = table.findAll("tr")

                    headers = [x.text for x in ths]
                    for j,tr in enumerate(trs[1:]):
                        tds  = tr.findAll('td')
                        vals = [x.text for x in tds]

                        if len(vals) == 3:
                            date,name,artist = vals
                            try:
                                date = getDateTime(", ".join([date, year]))
                            except:
                                print(vals)
                                print(", ".join([date, year]))

                                1/0
                        elif len(vals) == 1:
                            date = vals[0]
                            try:
                                date = getDateTime(", ".join([date, year]))
                            except:
                                print(vals)
                                1/0

                        if not isDate(date):
                            raise ValueError("Could not form date for {0}".format(date))

                        if data.get(artist) is None:
                            data[artist] = {}
                            print("Artist: {0}".format(artist))
                        if data[artist].get(name) is None:
                            data[artist][name] = {}
                            #print("\tSong: {0} ({1})".format(name, len(data[artist])))
                        if data[artist][name].get(category) is None:
                            data[artist][name][category] = []
                        data[artist][name][category].append([date])



                print("Found {0} artists".format(len(data)))


    for artist,artistData in data.items():
        for name,nameData in artistData.items():
            for category,categoryData in nameData.items():
                data[artist][name][category] = [min(categoryData),max(categoryData)]


    savedir  = join(basedir, "results")
    savename = join(savedir, "billboard-{0}.p".format(categoryType))
    saveFile(ifile=savename, idata=data, debug=True)

Artist: The Smashing Pumpkins
Artist: Colbie Caillat
Artist: Sum 41
Artist: Korn
Artist: Dave Matthews & Tim Reynolds
Artist: Nickelback
Artist: Atreyu
Artist: Chiodos
Artist: KT Tunstall
Artist: Foo Fighters
Artist: matchbox twenty
Artist: Kid Rock
Artist: Serj Tankian
Artist: Avenged Sevenfold
Artist: Angels & Airwaves
Artist: The Killers
Artist: OneRepublic
Found 17 artists
Artist: Radiohead
Artist: The Mars Volta
Artist: Jack Johnson
Artist: Panic! At The Disco
Artist: R.E.M.
Artist: Flight Of The Conchords
Artist: Portishead
Artist: Death Cab For Cutie
Artist: 3 Doors Down
Artist: Disturbed
Artist: Coldplay
Artist: Slipknot
Artist: Metallica
Artist: Snow Patrol
Artist: Soundtrack
Found 32 artists
Artist: The Fray
Artist: U2
Artist: Depeche Mode
Artist: Ben Harper And Relentless7
Artist: Green Day
Artist: Dave Matthews Band
Artist: Wilco
Artist: All Time Low
Artist: The Dead Weather
Artist: Kings Of Leon
Artist: Third Eye Blind
Artist: Skillet
Artist: Chevelle
Artist: Boys Like Gir

Artist: Peggy Scott-Adams
Artist: Jonny Lang
Artist: The Kenny Wayne Shepherd Band
Artist: B.B. King
Found 348 artists
Artist: Keb' Mo'
Found 349 artists
Artist: Albert King With Stevie Ray Vaughan
Found 350 artists
Artist: B.B. King & Eric Clapton
Found 351 artists
Artist: Double Trouble
Artist: Delbert McClinton
Artist: Buddy Guy
Artist: The Word
Found 355 artists
Artist: Etta James & The Roots Band
Artist: John Mayall And The Bluesbreakers
Artist: Joe Bonamassa
Artist: Shemekia Copeland
Artist: Susan Tedeschi
Found 360 artists
Artist: Etta James
Artist: Jim Belushi, Dan Aykroyd
Artist: The Peter Malick Group Featuring Norah Jones
Found 363 artists
Artist: Aerosmith
Artist: George Thorogood & The Destroyers
Found 365 artists
Artist: North Mississippi Allstars
Found 366 artists
Artist: The Derek Trucks Band
Artist: James Hunter
Artist: The Robert Cray Band
Artist: Maria Muldaur
Found 370 artists
Artist: Joan Armatrading
Artist: Robben Ford
Artist: Bettye LaVette
Artist: Stevie Ray Vau

Found 678 artists
Artist: Vince Guaraldi Trio
Artist: Billy Joel
Artist: ABBA
Found 681 artists
Artist: Creedence Clearwater Revival Featuring John Fogerty
Artist: Dean Martin
Artist: Mary J. Blige
Artist: The Countdown Kids
Artist: George Strait
Artist: tobyMac
Artist: Casting Crowns
Found 688 artists
Artist: Chris Tomlin
Artist: Tenth Avenue North
Artist: John Lennon
Artist: John Lennon & Yoko Ono
Artist: Michael W. Smith
Found 693 artists
Artist: Kari Jobe
Artist: Cage The Elephant
Artist: Amy Winehouse
Found 696 artists
Artist: Paul & Linda McCartney
Artist: Frank Sinatra
Found 698 artists
Artist: Chicago
Artist: The Postal Service
Artist: Blake Shelton
Artist: George Jones
Found 702 artists
Artist: Lindsey Stirling
Artist: Frankie Valli & The 4 Seasons
Artist: Boston
Artist: Robin Williams
Found 706 artists
Artist: Daryl Hall John Oates
Artist: Roger Waters
Artist: N.W.A
Artist: Paul McCartney
Found 710 artists
Artist: Prince And The Revolution
Artist: Juan Gabriel
Found 712 artis

Found 946 artists
Artist: Ron White
Artist: "Weird Al" Yankovic
Found 948 artists
Artist: Patton Oswalt
Artist: Nick Swardson
Found 950 artists
Artist: Kathy Griffin
Artist: Lewis Black
Artist: Mitch Hedberg
Artist: Jeff Dunham
Found 954 artists
Artist: Katt Williams
Artist: The Lonely Island
Artist: Jim Gaffigan
Artist: Steel Panther
Artist: Artie Lange
Found 959 artists
Artist: Aziz Ansari
Artist: Bo Burnham
Artist: Brian Posehn
Artist: Tom Lehrer
Artist: David Cross
Artist: Infant Sorrow
Artist: Dirt Nasty
Artist: Doug Benson
Artist: Fred Figglehorn
Found 968 artists
Artist: Louis C.K.
Artist: Daniel Tosh
Artist: Mike Birbiglia
Artist: Doug Stanhope
Found 972 artists
Artist: Kevin Hart
Artist: John Mulaney
Artist: Patrice O'Neal
Artist: Garfunkel And Oates
Artist: Them Idiots Featuring Bill Engvall, Jeff Foxworthy, Larry The Cable Guy
Artist: Joey Coco Diaz
Artist: Tenacious D
Artist: Jimmy Fallon
Artist: Thomas Miles aka Nephew Tommy
Artist: Demetri Martin
Artist: Tig Notaro
Artist

Artist: Brian Eno
Artist: Kygo
Artist: Flume
Artist: DJ Shadow
Artist: Clams Casino
Artist: DJ Mark Farina
Artist: ZHU
Artist: DJ Snake
Artist: Tycho
Artist: William Control
Artist: Empire Of The Sun
Found 1281 artists
Found 1281 artists
Artist: Marshmello
Artist: RL Grime
Artist: Robyn
Artist: Clean Bandit
Found 1285 artists
Found 1285 artists
Artist: Matisyahu
Artist: Ben Harper
Artist: T.I.
Artist: Keane
Artist: Dashboard Confessional
Artist: Thom Yorke
Artist: Tom Petty
Artist: Five For Fighting
Artist: Danity Kane
Artist: Diddy
Artist: Daughtry
Found 1296 artists
Artist: Timbaland
Artist: Traveling Wilburys
Artist: The White Stripes
Artist: Jimmy Eat World
Artist: Alicia Keys
Found 1301 artists
Artist: The Black Crowes
Artist: Counting Crows
Artist: Gavin DeGraw
Artist: Young Jeezy
Artist: David Cook
Found 1306 artists
Artist: Asher Roth
Found 1307 artists
Artist: B.o.B
Artist: Sara Bareilles
Artist: Kid Cudi
Found 1310 artists
Artist: Lupe Fiasco
Found 1311 artists
Artist: Ingrid

...

Artist: Timbaland Featuring Drake
Artist: Chris Brown Featuring Tyga & Kevin McCall
Found 5517 artists
Artist: Jamie Foxx Featuring Drake
Artist: Nicki Minaj Featuring Drake
Artist: Chris Brown Featuring Lil Wayne & Busta Rhymes
Artist: Kelly Rowland Featuring Lil Wayne
Artist: DJ Khaled Featuring Drake, Rick Ross & Lil Wayne
Artist: Big Sean Featuring Kanye West & Roscoe Dash
Found 5523 artists
Artist: Wale Featuring Miguel
Artist: Drake Featuring Nicki Minaj
Artist: Drake Featuring Lil Wayne
Artist: Kanye West, Big Sean, Pusha T, 2 Chainz
Artist: 2 Chainz Featuring Drake
Found 5528 artists
Artist: A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar
Artist: Wale Featuring Tiara Thomas Or Rihanna
Artist: J. Cole Featuring Miguel
Artist: Drake Featuring Majid Jordan
Found 5532 artists
Artist: Chris Brown Featuring Lil Wayne & French Montana Or Too $hort Or Tyga
Artist: ScHoolboy Q Featuring BJ The Chicago Kid
Artist: Jeremih Featuring YG
Artist: Rich Gang Featuring Young Thug & Rich H

Artist: Mana Featuring Shakira
Artist: Pitbull Featuring Gente de Zona
Artist: Nicky Jam & Enrique Iglesias
Artist: Wisin + Carlos Vives Featuring Daddy Yankee
Artist: Gente de Zona Featuring Marc Anthony
Artist: Ricky Martin Featuring Yotuel
Artist: Farruko Featuring Shaggy & Nicky Jam
Artist: La Adictiva Banda San Jose de Mesillas
Artist: Alvaro Soler Featuring Jennifer Lopez
Found 5801 artists
Artist: Carlos Vives & Shakira
Artist: Maluma Featuring Yandel
Artist: IAmChino Featuring Pitbull, Yandel & Chacal
Artist: Mana & Nicky Jam
Artist: Ricky Martin Featuring Maluma
Artist: J Balvin Featuring Pharrell Williams, BIA & Sky
Artist: Zion & Lennox Featuring J Balvin
Artist: Shakira Featuring Maluma
Artist: Farruko Featuring Ky-Mani Marley
Artist: Don Omar & Zion & Lennox
Found 5811 artists
Artist: Maluma Featuring Fifth Harmony Or Don Omar & Wisin
Artist: Enrique Iglesias Featuring Descemer Bueno, Zion & Lennox Or Sean Paul
Artist: Wisin Featuring Ozuna
Artist: Yandel Featuring Wisin
A

Artist: Bobby Day
Artist: Cozy Cole
Artist: Clyde McPhatter
Artist: Jackie Wilson
Found 5968 artists
Artist: James Brown And The Famous Flames
Artist: Lloyd Price and His Orchestra
Artist: Fats Domino
Artist: The Coasters
Artist: Phil Phillips With The Twilights
Artist: Della Reese
Artist: The Spacemen
Found 5975 artists
Artist: Bill Black's Combo
Artist: Dinah Washington & Brook Benton
Artist: Buster Brown
Artist: Bobby Marchan
Artist: Dinah Washington
Artist: Hank Ballard And The Midnighters
Artist: Jerry Butler
Found 5982 artists
Artist: The Miracles (featuring Bill "Smokey" Robinson)
Artist: Bobby Bland
Artist: Ben E. King
Artist: Pips
Artist: The Jive Five With Joe Rene And Orchestra
Artist: Lee Dorsey
Found 5988 artists
Artist: Barbara George
Artist: King Curtis And The Noble Knights
Artist: Dee Dee Sharp
Artist: Barbara Lynn
Artist: Booker T. & The MG's
Artist: The Contours
Artist: "Little Esther" Phillips
Found 5995 artists
Artist: Barbara Lewis
Artist: Martha & The Vandellas
A

Artist: Junior M.A.F.I.A. Featuring The Notorious B.I.G.
Artist: Nonchalant
Artist: Do Or Die (Featuring Twista)
Artist: Lil' Kim Featuring Puff Daddy
Found 6217 artists
Artist: Magoo And Timbaland
Artist: Puff Daddy & The Family (Feat. The Notorious B.I.G. & Mase)
Found 6219 artists
Artist: Lord Tariq & Peter Gunz
Artist: Sylk-E. Fyne Featuring Chill
Artist: The Lox [Featuring DMX & Lil' Kim]
Artist: Luke Featuring No Good But So Good
Artist: Master P Featuring Sons Of Funk
Artist: Puff Daddy Featuring Jimmy Page
Artist: Mase Featuring Puff Daddy
Artist: Ice Cube Featuring Mr. Short Khop
Artist: Mo Thugs Family Featuring Bone Thugs-N-Harmony
Found 6228 artists
Artist: Cool Breeze Featuring OutKast, Goodie Mob & Witchdoctor
Artist: Silkk The Shocker Featuring Mystikal
Artist: JT Money Featuring Sole
Artist: Sporty Thievz Featuring Mr. Woods
Artist: Naughty By Nature Featuring Zhane
Artist: Warren G Featuring Mack 10
Artist: Sole Featuring JT Money & Kandi
Found 6235 artists
Artist: Big

Artist: Victor Manuelle & Elvis Crespo
Found 6366 artists
Artist: Anthony
Found 6367 artists
Artist: Eddy Herrera
Found 6368 artists
Artist: Brenda K. Starr Con Tito Nieves & Victor Manuelle
Artist: Area 305
Found 6370 artists
Found 6370 artists
Artist: Los Toros Band
Artist: Tito Nieves
Found 6372 artists
Artist: Tito Nieves Featuring India, Nicky Jam & K-Mil
Artist: Jimena
Artist: Ismael Miranda Featuring Cheka & Andy Montanez
Artist: Arthur Hanlon Featuring Tito Nieves
Artist: Baby Ranks, Daddy Yankee, Tonny Tun Tun, Wisin & Yandel & Hector
Artist: Aventura Featuring Don Omar
Artist: Chichi Peralta With Joe Vasconcelos
Found 6379 artists
Artist: Frank Reyes
Artist: Don Omar, Gilberto Santa Rosa
Found 6381 artists
Artist: Wisin & Yandel Featuring Los Vaqueros
Artist: Victor Manuelle Duet With Don Omar
Found 6383 artists
Artist: NG2
Artist: Hector Acosta "El Torito"
Artist: El Chaval de la Bachata
Artist: MJ
Artist: Charlie Cruz
Artist: Adolescent's Orquesta
Artist: Jorge Celedon & Ji

# Check For Misnames

In [31]:
ma = multiartist(cutoff=0.9, discdata=discdata)

In [33]:
print(ma.getArtistNames('Lipps, Inc.') == {'Lipps, Inc.': ['159617', '26641', '101850']})
print(ma.getArtistNames('Bob Wills & His Texas Playboys') == {'Bob Wills & His Texas Playboys': ['670000','786114','1134146','804668','804679','1004309','935875','935907']})
print(ma.getArtistNames('Brad Paisley Duet With Carrie Underwood', debug=False) == {'Carrie Underwood': ['1011680'], 'Brad Paisley': ['313755']})
print(ma.getArtistNames('Ray Charles With Willie Nelson', debug=False) == {'Willie Nelson': ['249449'], 'Ray Charles': ['521963', '30552']})

True
True
True
True


# Album Data

In [16]:
savename = join(savedir, "billboard.p")
albumData = getFile(join(savedir, "billboard-album.p"))
#chartdata = getFile(savename, debug=True)

In [18]:
for artist,artistData in albumData.items():
    print(artist)
    for album in artistData.keys():
        print("\t",album)

The Smashing Pumpkins
	 Zeitgeist
	 Oceania
	 Siamese Dream
	 Mellon Collie And The Infinite Sadness
Colbie Caillat
	 Coco
	 Breakthrough
	 All Of You
Sum 41
	 Underclass Hero
Korn
	 Untitled
	 Korn III: Remember Who You Are
	 The Serenity Of Suffering
	 Follow The Leader
	 The Path Of Totality
	 The Paradigm Shift
	 Korn
Dave Matthews & Tim Reynolds
	 Live At Radio City
	 Live In Las Vegas
Nickelback
	 All The Right Reasons
	 Dark Horse
	 Here And Now
	 No Fixed Address
	 Silver Side Up
	 The Long Road
	 Feed The Machine
Atreyu
	 Lead Sails Paper Anchor
	 The Curse
	 A Death-Grip On Yesterday
Chiodos
	 Bone Palace Ballet
KT Tunstall
	 Drastic Fantastic
Foo Fighters
	 Echoes, Silence, Patience & Grace
	 Wasting Light
	 Sonic Highways (Soundtrack)
	 Concrete And Gold
	 Greatest Hits
matchbox twenty
	 Exile On Mainstream
	 North
Kid Rock
	 Rock N Roll Jesus
	 Born Free
	 Devil Without A Cause
	 Sweet Southern Sugar
	 First Kiss
Serj Tankian
	 Elect The Dead
Avenged Sevenfold
	 Avenged Se

	 Bachata # 1s
	 Bachata # 1's: Vol. 2
	 Sergio George Presents: Salsa Giants
	 Cuba y Puerto Rico Son...
	 Latin Hits 2016: Club Edition
	 Global Meditation
	 Celtic Christmas
	 Red Hot + Rio
	 Celtic Christmas II
	 Celtic Christmas III
	 Celtic Christmas-Silver Anniversary Edition
	 Celtic Circle
	 The Very Best Of Celtic Christmas
	 The Celtic Circle 2
	 Pure Irish: The Ultimate St. Patrick's Day Celebration
	 Big Blue Ball
	 Paris Magnifique
	 Playing For Change: Songs Around The World
	 World Is Africa
	 World Is China
	 Mele O Hawaii: Songs Of Hawaii
	 Playing For Change: PFC 2: Songs Around The World
	 Musique Pop De Paris
	 The Spirit Of Ireland
	 Red Hot + Fela
	 It's A Scream How Levine Does The Rhumba: The Latin-Jewish Musical Story
	 Arrivederci Italy
	 One Love, One Rhythm: The 2014 FIFA World Cup Official Album
	 Brazil: Music Is The Pulse
She & Him
	 Volume Two
	 A Very She & Him Christmas
	 Volume 3
	 Classics
Jonsi
	 Go
MGMT
	 Congratulations
	 Oracular Spectacular
	 L

	 Mutineers
	 Life In Slow Motion
	 White Ladder
Crosby, Stills, Nash & Young
	 CSNY 1974 (Deluxe)
	 CSNY 1974
Trampled By Turtles
	 Wild Animals
	 Palomino
	 Stars And Satellites
	 Life Is Good On The Open Road
Shovels & Rope
	 Swimmin' Time
	 O' Be Joyful
Justin Furstenfeld
	 Songs From An Open Book
Lucinda Williams
	 Down Where The Spirit Meets The Bone
	 The Ghosts Of Highway 20
	 Lucinda Williams
	 West
	 Little Honey
	 Blessed
You+Me
	 rose ave.
Damien Rice
	 My Favourite Faded Fantasy
	 O
George Ezra
	 Wanted On Voyage
Mark Knopfler
	 Tracker
	 Golden Heart
Lord Huron
	 Strange Trails
	 Vide Noir
The Tallest Man On Earth
	 Dark Bird Is Home
Sawyer Fredericks
	 The Voice: The Complete Season 8 Collection
Dawes
	 All Your Favorite Bands
	 We're All Gonna Die
	 Passwords
James Taylor
	 Before This World
	 Greatest Hits
Jason Isbell
	 Something More Than Free
Nathaniel Rateliff & The Night Sweats
	 Nathaniel Rateliff & The Night Sweats
	 Tearing At The Seams
Ben Rector
	 Brand New
	

	 Christina Aguilera
	 Back To Basics
	 My Kind Of Christmas
	 Mi Reflejo
Matthew Good Band
	 Beautiful Midnight
Lou Bega
	 A Little Bit Of Mambo
Rage Against The Machine
	 The Battle Of Los Angeles
	 Rage Against The Machine
Savage Garden
	 Affirmation
Santana
	 Supernatural
	 All That I Am
	 Santana IV
	 Corazon
'N Sync
	 No Strings Attached
	 Celebrity
	 Home For Christmas
Toni Braxton
	 The Heat
	 Toni Braxton
	 Secrets
	 Pulse
Eminem
	 The Marshall Mathers LP
	 The Eminem Show
	 Encore
	 Curtain Call: The Hits
	 Relapse
	 Recovery
	 The Marshall Mathers LP 2
	 Revival
	 Kamikaze
	 The Slim Shady LP
The Beatles
	 1
	 Love
	 Sgt. Pepper's Lonely Hearts Club Band
	 The Beatles 1967-1970
	 Abbey Road
	 Magical Mystery Tour (Soundtrack)
	 Let It Be... Naked
	 The Beatles [White Album]
	 On Air: Live At The BBC Volume 2
	 Long Tall Sally (EP)
	 The Beatles In Stereo
Jennifer Lopez
	 J.Lo
	 Como Ama Una Mujer
	 A.K.A.
	 J To Tha L-O! The Remixes
Shaggy
	 Hotshot
	 Boombastic
	 Midnight L

	 untitled unmastered.
	 DAMN.
	 good kid, m.A.A.d city
	 Section.80
Yoan
	 YOAN
Shawn Mendes
	 Handwritten
	 Illuminate
	 Shawn Mendes
Meek Mill
	 Dreams Worth More Than Money
	 Championships
	 Dreams And Nightmares
Dr. Dre
	 Compton
	 The Chronic
	 Dr. Dre -- 2001
The Weeknd
	 Beauty Behind The Madness
	 Starboy
	 My Dear Melancholy, (EP)
	 Kiss Land
	 Trilogy
Drake & Future
	 What A Time To Be Alive
Bernard Adamus
	 Sorel Soviet So What
Hedley
	 Hello
Zayn
	 Mind Of Mine
Lukas Graham
	 Lukas Graham
Prince
	 The Very Best Of Prince
	 ART OFFICIAL AGE
	 Piano & A Microphone 1983
	 3121
	 Planet Earth
Pentatonix
	 A Pentatonix Christmas
	 PTXmas (EP)
	 That's Christmas To Me
	 Pentatonix
	 Christmas Is Here!
	 PTX: Vol. II
Migos
	 Culture
	 Culture II
Big Sean
	 I Decided.
	 Dark Sky Paradise
	 Hall Of Fame
	 Finally Famous
Future
	 FUTURE
	 HNDRXX
	 DS2
	 EVOL
	 Honest
The Chainsmokers
	 Memories...Do Not Open
	 Collage (EP)
	 Sick Boy
Harry Styles
	 Harry Styles
Calvin Harris
	 Funk 

...

	 Lyfe 268-192
	 The Phoenix
	 Lyfe Change
Armor For Sleep
	 What To Do When You Are Dead
Celtic Woman
	 Celtic Woman
	 A Christmas Celebration
	 A New Journey
	 The Greatest Journey: Essential Collection
	 Songs From The Heart
	 Lullaby
	 Believe
	 Home For Christmas
	 Emerald: Musical Gems
	 Christmas
	 Destiny
ZOEgirl
	 Room To Breathe
Corrosion Of Conformity
	 In The Arms Of God
	 No Cross No Crown
Mindless Self Indulgence
	 You'll Rebel To Anything
Slightly Stoopid
	 Closer To The Sun
	 Everyday Life, Everyday People
Jeff Bates
	 Rainbow Man
Bobby Pinson
	 Man Like Me
Dark New Day
	 Twelve Year Silence
Dredg
	 Catch Without Arms
DevilDriver
	 The Fury Of Our Maker's Hand
The Black Dahlia Murder
	 Miasma
Shooter Jennings
	 Put The O Back In Country
Dope
	 American Apathy
Josh Kelley
	 Almost Honest
Against Me!
	 Searching For A Former Clarity
	 Transgender Dysphoria Blues
RBD
	 Rebelde
	 Nuestro Amor
	 Celestial
	 Empezar Desde Cero
God Forbid
	 IV: Constitution Of Treason
Ana Gabr

Last In Line
	 Heavy Crown
School Of Seven Bells
	 SVIIB
Lapsley
	 Long Way Home
Penny & Sparrow
	 Let A Lover Drown You
	 Wendigo
Saddleback Worship
	 F I R S T (EP)
Domo Genesis
	 Genesis
Charles Bradley
	 Changes
Sam Beam & Jesca Hoop
	 Love Letter For Fire
Jamesthemormon
	 I'm Not A Rapper (EP)
Fallujah
	 Dreamless
ANOHNI
	 HOPELESSNESS
Nothing
	 Tired Of Tomorrow
	 Dance On The Blacktop
Katatonia
	 The Fall Of Hearts
PUP
	 The Dream Is Over
Vic Mensa
	 There's Alot Going On (EP)
Nails
	 You Will Never Be One Of Us
DOROTHY
	 ROCKISDEAD
Sara Watkins
	 Young In All The Wrong Ways
Invent, Animate
	 Stillworld
Cane Hill
	 Smile
	 Too Far Gone
Jason Richardson
	 I
Nick Fradiani
	 Hurricane
Upchurch
	 Chicken Willie
Gemini Syndrome
	 Memento Mori
Joseph
	 I'm Alone, No You're Not
Deborah Silver
	 The Gold Standards
Hamilton Leithauser + Rostam
	 I Had A Dream That You Were Mine
S U R V I V E
	 RR7349
Chuck Murphy
	 Peace Be With You
STRFKR
	 Being No One, Going Nowhere
dodie
	 Intertwine

	 Cama Incendiada
Alejandro Fernandez
	 Me Estoy Enamorando
	 Entre Tus Brazos
	 Dos Mundos: Evolucion
	 Confidencias
	 Confidencias Reales
	 Rompiendo Fronteras
	 Origenes
	 A Corazon Abierto
	 Viento A Favor
	 De Noche: Clasicos A Mi Manera...
	 Dos Mundos: Tradicion
Elvis Crespo
	 Suavemente
	 Pintame
	 The Remixes
	 Wow Flash!
	 Urbano
	 Saborealo
Los Temerarios
	 En La Madrugada Se Fue
	 Historia Musical
	 Una Lagrima No Basta
	 Tributo Al Amor
	 Veintisiete
	 Recuerdos Del Alma
	 Si Tu Te Vas
	 De Lo Nuevo Lo Mejor
	 Mi Vida Eres Tu
	 Como Te Recuerdo
	 15 Exitos Para Siempre
	 Regalo De Amor
	 Sueno De Amor
	 30 Aniversario
Ricardo Arjona
	 Galeria Caribe
	 5to Piso
	 Poquita Ropa
	 Independiente
	 Viaje
	 Quien Dijo Ayer
	 Metamorfosis: En Vivo
	 Apague La Luz y Escuche
	 Circo Soledad
Vicente Fernandez
	 Historia De Un Idolo Vol. 1
	 Historia De Un Idolo
	 Para Siempre
	 Primera Fila
	 El Hombre Que Mas Te Amo
	 Muriendo de Amor
	 Por Tu Maldito Amor
	 Entre El Amor Y Yo
	 Y L

	 We Are Family
Donna Summer
	 Bad Girls
Kool & The Gang
	 Ladies' Night
	 Something Special
The Whispers
	 The Whispers
	 Love Is Where You Find It
Jermaine Jackson
	 Let's Get Serious
	 Jermaine Jackson
Cameo
	 Cameosis
	 She's Strange
	 Word Up
Tom Browne
	 Love Approach
Zapp
	 Zapp
The Jacksons
	 Triumph
The Gap Band
	 The Gap Band III
	 Gap Band Iv
	 Gap Band VI
Yarbrough & Peoples
	 The Two Of Us
Ray Parker Jr. & Raydio
	 A Woman Needs Love
Rick James
	 Street Songs
	 Cold Blooded
Jarreau
	 Breakin' Away
Roger
	 The Many Facets Of Roger
Luther Vandross
	 Never Too Much
	 Forever, For Always, For Love
	 Busy Body
	 The Night I Fell In Love
	 Give Me The Reason
	 Any Love
	 Power Of Love
	 Dance With My Father
Skyy
	 Skyy Line
Bobby Womack
	 The Poet
Shalamar
	 Friends
Atlantic Starr
	 Brilliance
Ray Parker Jr.
	 The Other Woman
Dazz Band
	 Keep It Live
Evelyn "Champagne" King
	 Get Loose
One Way
	 Lady
Tina Turner
	 Private Dancer
Maze Featuring Frankie Beverly
	 Can't Stop The Lo

	 Corridos Time, Season One: Soy Parrandero
Los Angeles Azules
	 Confesiones De Amor
	 Historia Musical
Los Super Seven
	 Los Super Seven
Juan Gabriel Con Banda El Recodo
	 Juan Gabriel Con La Banda...El Recodo!!!
Los Tri-o
	 Nuestro Amor
Los Huracanes del Norte
	 Norteno 2000
	 En Que Trabaja El Muchacho
Alicia Villarreal
	 Soy Lo Prohibido
	 La Jefa
Liberacion
	 Historia Musical
Vicente Y Alejandro Fernandez
	 En Vivo: Juntos Por Ultima Vez
Marco Antonio Solis & Joan Sebastian
	 Dos Grandes
Los Horoscopos de Durango
	 Y Seguimos Con Duranguense!!!
	 Antes Muertas Que Sencillas
	 Viejitas Pero Buenas... Pa' Pistear
Beto y Sus Canarios
	 Ardientes
Graciela Beltran
	 Rancherisimas Con Banda
Ramon Ayala y Sus Bravos del Norte
	 Antologia De Un Rey
Jose Alfredo Jimenez
	 La Historia Del Rey
Diana Reyes
	 Las No. 1 De La Reina
	 Insatisfecha
Brazeros Musical de Durango
	 Rosas Rojas
Javier Solis
	 La Historia De Javier Solis
Pesado
	 Piensame Un Momento
	 Mi Promesa
	 Directo 93-13, Vol. 1

# Show Data

In [34]:
togets = {}
combos = []

savedir  = join(basedir, "results")
savename = join(savedir, "billboard.p")
chartdata = getFile(savename, debug=True)
chartArtists = sorted(chartdata.keys(), reverse=True)

import json
prevs = json.load(open("prevs.json", "r"))
artPrev = json.load(open("../discogs/prevs.json", "r"))
prevs.update(artPrev)
json.dump(prevs, open("prevs.json", "w"))
print("There are {0} artists previously searched.".format(len(prevs)))

ma = multiartist(cutoff=0.9, discdata=discdata, exact=False)

from random import shuffle
print("There are {0} chart artists".format(len(chartArtists)))
shuffle(chartArtists)
for i,artist in enumerate(chartArtists):
    if prevs.get(artist) is not None:
        continue
    retval = ma.getArtistNames(artist)
    if retval is None:
        togets[artist] = artist
    else:
        for name,value in retval.items():
            if prevs.get(name) is not None:
                continue
            if value is None:
                togets[name.upper()] = artist
                print("{0: <3}{1: <40}{2: <60}{3}".format(len(togets),name,artist,retval))
                print('\t---->',i,'/',len(chartArtists))

    
    if len(togets) > 50:
        print("\n\n\n\n")
        for name,artist in togets.items():
            print("art.searchDiscogForArtist(\"{0}\")  ## {1}".format(name, artist))
        break

Loading data from /Users/tgadfort/Documents/code/charts/results/billboard.p
  --> This file is 1.1MB.
Loading /Users/tgadfort/Documents/code/charts/results/billboard.p
There are 489 artists previously searched.
There are 6554 chart artists
1  London Phil. Orch. Palmer               London Phil. Orch. (Palmer)                                 {'London Phil. Orch. Palmer': None}
	----> 36 / 6554
2  The Sound Of Victory Fellowship Choir   Vincent Bohanan And The Sound Of Victory Fellowship Choir   {'The Sound Of Victory Fellowship Choir': None, 'Vincent Bohanan': ['4377432']}
	----> 58 / 6554
3  Amanda Perez                            Amanda Perez                                                {'Amanda Perez': None}
	----> 64 / 6554
4  Dcup                                    Yolanda Be Cool & Dcup                                      {'Dcup': None, 'Yolanda Be Cool': ['1561295']}
	----> 65 / 6554
5  A. Karl                                 Queen Latifah, D. Parton, K. Palmer, J. Jordan, A. 

45 Pavarotti Metha                         Carreras, Domingo, Pavarotti (Metha & Levine)               {'Domingo': ['270225', '3114727', '696253', '152272', '13970'], 'Pavarotti Metha': None, 'Carreras': ['462603', '2785188', '820814', '3209346', '1568874', '3314443', '3413497'], 'Levine': ['121607', '353608', '837627', '278195', '1189702', '70219', '34252', '696260', '575269', '490530', '789324', '276888', '538470', '156489', '792992', '267494', '1388566']}
	----> 1032 / 6554
46 The Dubplates                           The Dubplates                                               {'The Dubplates': None}
	----> 1041 / 6554
47 BT featuring Tori Amos                  BT featuring Tori Amos                                      {'BT featuring Tori Amos': None}
	----> 1047 / 6554
48 Isha CoCo                               Dave Aude Featuring Isha CoCo                               {'Isha CoCo': None, 'Dave Aude': ['2867']}
	----> 1058 / 6554
49 Su Banda Tierra Santa                   El Coyote

###### artists = [x.strip() for x in vals.split("\n")]
artists = [x for x in artists if len(x) > 0]

In [ ]:
Pitbull Featuring Ne-Yo, Afrojack & Nayer
Drake Featuring Kanye West, Lil Wayne & Eminem

In [ ]:
def testName(name, discdata):
    if discdata.get(name) is not None:
        return discdata[name]
    return name

def findName(name, discArtists, threshold):
    retval = findNearest(name, discArtists, 1, threshold)
    if len(retval) == 0:
        return name
    return retval
    

def splitArtist(name):
    names = [name]
    names = [[x.strip() for x in y.split(" Featuring ")]
results = {}
    
for name in artists:
    if " & " in name:
        retvals = [testName(x, discdata) for x in 
        retvals = [findName(x, discArtists, 0.95) if not isinstance(x, list) else x for x in retvals]
        results[name] = retvals        

In [ ]:
togets = []
for k,v in results.items():
    for val in v:
        if not isinstance(val, list):
            togets.append("art.searchDiscogForArtist(\"{0}\")".format(val))
            
for toget in set(togets):
    print(toget)

In [ ]:
tmp = [['5070865', '90037', '4135543', '1507065'], ['Katy XXX Perry', ['1201210']]]

In [ ]:
tmp

In [46]:
nameids = {}
namerefs = {}
for name,nameid in iddata.items():
    artist = name
    if name.endswith(")"):
        artist = None
        for x in [-3,-4,-5]:
            if artist is not None:
                continue
            if abs(x) > len(name):
                continue
            if name[x] == "(":
                try:
                    val = int(name[(x+1):-1])
                    artist = name[:x].strip()
                except:
                    continue
          
        if artist is None:
            artist = name

    ref = refdata[name]
            
    if nameids.get(artist) is None:
        nameids[artist] = {}
    nameids[artist][nameid] = 1
    
    if namerefs.get(artist) is None:
        namerefs[artist] = {}
    namerefs[artist][ref] = 1
    

nameids = {k: list(v.keys()) for k,v in nameids.items()}
namerefs = {k: list(v.keys()) for k,v in namerefs.items()}

NameError: name 'iddata' is not defined